# Архитектуры CNN

Мы рассмотрели базовые компоненты из которых состоят современные сверточные нейронные сети, а так же техники их обучения.

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_1.png"  width="700">

На этом занятии рассмортим какие именно модели можно построить на основе этих компонент. 




<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_2.png"  width="700">

Условия: http://image-net.org/challenges/LSVRC/2017/

Загрузка с [официального сайта](http://image-net.org/download) недоступна.


Можно загрузить данные с [Kaggle](https://www.kaggle.com/c/imagenet-object-localization-challenge/data)

Однако архив занимает порядка 156Gb и не поместится на диск Colab. 







In [ ]:
# Full list of labels 
#! wget "https://gist.githubusercontent.com/yrevar/942d3a0ac09ec9e5eb3a/raw/238f720ff059c1f82f368259d1ca4ffa5dd8f9f5/imagenet1000_clsidx_to_labels.txt"
#! wget 'https://gist.githubusercontent.com/yrevar/6135f1bd8dcf2e0cc683/raw/d133d61a09d7e5a3b36b8c111a8dd5c4b5d560ee/imagenet1000_clsid_to_human.pkl'
! wget 'https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json'

# Tiny Image-Net
#урезанная версия датасета подготовленная в стенфордском университете.
#!wget 'http://cs231n.stanford.edu/tiny-imagenet-200.zip'
#!unzip tiny-imagenet-200.zip

! git clone https://github.com/ajschumacher/imagen.git



Загрузили категории

In [ ]:
import json

with open('imagenet_class_index.json') as f:
  imagenet_labels = json.load(f)
print(imagenet_labels)

Посмотрим на картинки

In [ ]:
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (25,15)
plt.tight_layout()

images = glob('imagen/imagen/*.jpg')

def show(img,label1,num,label2=""):
    ax = plt.subplot(2, 3,num+1)
    
    plt.imshow(img)
    #plt.axis('off')
    plt.title(label1)
    ax.set_xlabel(label2)
  

for i, path in enumerate(images[:6]):
  im = Image.open(path)
  name = path.split("_")[2]
  show(im,name,i,"")

Как упомяналось на первой лекции современный бум нейростевых технологий начался в 2012 году когда AlexNet большим отрывом от конкурентов победила в ImageNet

## AlexNet


создатели: Alex Krizhevsky, Ilya Sutskever, Geoffrey E. Hinton
**University of Toronto**


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_3.png"  width="850">


Особенности
- Большие фильтры на первых слоях
- Обучалась на двух видеокартах  GTX580 ( ... )
- Уменьшение размерности
- 2 Полносвязанных слоя в конце

https://neurohive.io/ru/vidy-nejrosetej/alexnet-svjortochnaja-nejronnaja-set-dlja-raspoznavanija-izobrazhenij/

https://papers.nips.cc/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf

ImageNet Classification with Deep ConvolutionalNeural Networks



В AlexNet есть все компоненты, которые мы рассматривали ранее. Её архитектура состоит из пяти свёрточных слоёв, между которыми располагаются pooling-слои и слои нормализации, а завершают нейросеть три полносвязных слоя.


На схеме архитектуры все выходные изображения делятся на два одинаковых участка — это связано с тем, что нейросеть обучалась на старых GPU GTX580, у которых было всего 3 ГБ видеопамяти. Для обработки использовались две видеокарты, чтобы параллельно выполнять операции над двумя частями изображения.
Изначально на вход подаётся фотография размером 227×227×3, и размер свёрточных фильтров первого слоя — 11×11. Всего применяется 96 фильтров с шагом 4.

Пространственные размеры изображения сначала довольно сильно сжимаются, затем постепенно увеличивается число фильтров. В результате ширина и высота этого уже не изображения, а набора активаций признаков, сильно уменьшаются, после чего оно поступает на два полносвязных своя, где количество весов уже довольно большое.


AlexNet не получится использовать для классификации cifar10, потому что если начать так агрессивно уменьшать изображение размером 32Х32 px, то в определенный момент в него просто не поместится следующий фильтр, который нужно применить, и изображение просто исчезнет.

Структура некоторых (особенно старых) сетей, заточенных под ImageNet, напрямую зависит от размера изображений: если соотношение сторон позволит фильтрам поместиться, проблем не возникнет,  если же оно меньше и в какой-то момент размер уменьшится до 2Х2 или 1Х1, то фильтр 3Х3 просто не сработает. В современных сетях есть слой, который позволяет решить эту проблему.



На тот момент такая архитектура показала прорывную точность (ошибка упала с 20% до 15.4%).




<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_4_1.png"  width="750">

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_4_2.png"  width="850">


Также стоит упомянуть о нескольких важных параметрах нейросети, необычных на тот момент:


* AlexNet — практически первая архитектура, в которой применяется нелинейность ReLU. Ранее использовались сигмоидальные функции, которые работали медленнее;

* Используется собственная нормализация (не столь универсальная, как Batch-нормализацию) с отдельными слоями. Как выяснилось позднее, они не дают значительного улучшающего эффекта и поэтому не распространены в современных архитектурах;

* На этапе предварительной обработки используется очень большое дополнение данных (аугментация);

* Dropout 0.5 (то есть при регуляризации отсеивается половина нейронов);

* Размер пакета — 128;

* SGD Momentum 0.9 (как показывают сегодняшние эксперименты, это не плохой вариант, но чтобы обучение сходилось, им требовалась с помощью эвристики ниже периодически обучать Learning rate);

* Скорость обучения — 1e−2, снижается в 10 раз вручную, если точность в какой-то момент перестаёт расти;

* Встречаются следующие рекомендации — изначально обучить сеть <font color=red >N(?)</font>, а затем добиться дополнительной точности в 2%, то можно вернуться к SGD Momentum и аккуратно поработать с ним с маленьким шагом;

* Затухание весов L2 — 5e−4;

* В архитектуре используется ансамбль из 7 CNN — это позволило снизить процент ошибок с 18,2% до 15,4%. (Ансамбль моделей — это когда обучается несколько моделей, а результат считается по среднему значению. Здесь используются 7 сетей, результат усредняется. Таким образом достаточно сильно снижается ошибка).


Сравним Pytorch реализацию с оригинальной. В чем отличия?

In [ ]:
from torchvision import models,utils
alexnet = models.alexnet(pretrained=True)
print(alexnet)

Проверим как работает. Почему так плохо?

In [ ]:

import torchvision.transforms.functional as F
import torch


def img2tensor(img):
  t = F.to_tensor(img)
  #t = F.normalize(t, (0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
  return t

def catId2names(nums):
  titles = []
  for num in nums:
    titles.append(imgenet_labels[str(num.item())][1])
  return ", ".join(titles)
  
for i, path in enumerate(images[:6]):
  im = Image.open(path)
  tensor = img2tensor(im)
  out = alexnet(tensor.unsqueeze(0)) # Add batch dimension
  labels_num = torch.argsort(out[0]) # Ascending order
  predicted = catId2names(labels_num[-5:]) # Top 5
  titles = []
  for num in labels_num[-5:]:
    titles.append(imagenet_labels[str(num.item())][1])
  name = path.split("_")[2]
  show(im,name,i,predicted)


## ZFnet


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_5.png"  width="850">


Тюнингованный AlexNet

Matthew D. Zeilerzeiler@cs.nyu.eduDept. of Computer Science, Courant Institute, Rob Fergusfergus@cs.nyu.eduDept. of Computer Science, **Courant Institute, New York Universit**


В 2013 году выиграла соревнования созданная учеными из Йорского университета нейросеть ZFnet, достигнув результата 11.7% — в ней AlexNet использовалась в качестве основы, но с изменёнными параметрами и слоями.

Отличия от AlexNet небольшие:
* Немного поменялись размеры фильтров (было 11, стало 7);
* Увеличилось общее количество фильтров;

В целом и количество слоев, и общая структура сети, когда слои свертки и пулинга перемежаются друг с другом, а затем идут два полносвязных слоя, сохранились.


## VGGNet

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_6.png"  width="850">

Karen Simonyan and Andrew Zisserman

Visual Geometry Group - **Oxford**

https://www.robots.ox.ac.uk/~vgg/research/very_deep/


* Появление "стандартных" блоков внутри модели
* Свертки 3x3





В 2014-ом году в Оксфорде была разработана модель VGGNet. Сеть получилась более точной и более глубокой.

На слайде выше изображены сети AlexNet и две версии VGG-16 и VGG-19 с 16 и 19 слоями соответственно. На соревнованиях победила более глубокая VGG19, достигнув более чем в два раза лучшего результата по сравнению с AlexNet.

Несколько фактов об архитектуре VGGNet:

* Нейросеть заняла 2 место в задаче классификации и 1 место в локализации на соревновании ImageNet (при локализации необходимо не только классифицировать объект, но и обвести его в ограничивающие рамки);

* Процедура обучения такая же, как у AlexNet;

* Слои нормализации отсутствуют.

В прикладных задачах обычно используются архитектуры VGG16 или VGG19 (VGG19 работает лучше, но расходует больше памяти).
Особенности данной архитектуры: все сверточные слои имеют фильтры с рецептивным полем размера 3×3, они объединены в блоки, состоящие из некоторого количества сверток с разным (постепенно увеличивающимся) количеством фильтров. Затем идут слои пулинга.
Появление "стандартных" блоков внутри модели - одно из важных нововведений. Идея базового блока внутри сети будет достаточно широко использоваться дальше.


Чтобы понять, в чем смысл этих усовершенствований, нужно оценить количество памяти для хранения весов и количество операций, которые требуется выполнить при прямом прохождении сигнала через сеть.

### Ресурсы

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_7.png"  width="850">

*Пример оценки (на AlexNet)*

Чем больше особенностей попадает в рецептивное поле свертки, тем к большим обобщениям она должна быть способна. Несмотря на это, авторы сети решили отказаться от использования сверток с большим рецептивным полем в пользу скорости потому, что оказалось, что размер можно заменить количеством, что 2 маленьких фильтра работают как один большой или даже лучше.

Основная причина, по которой они начали об этом думать - это увеличение сложности. **Большее количество слоев требует больше памяти, весов и вычислительных мощностей**.

На изображении выше показано, из чего они <font color=red >(кто? вычислительные ресурсы?)</font> складываются.

На слайде выше изображен один слой. Тензор красного цвета - это вход, его мы не считаем.

Первый параметр - **количество элементов на выходе слоя**. Синий тензор - это активации, для которых требуется память во время работы модели. Чтобы понять, сколько нужно памяти, чтобы хранить активации, которые мы получаем, нужно умножить количество фильтров на пространственные размеры изображения и на количество байт, которые требуются для хранения одного элемента.

**Количество весов (параметров)** - это пространственные размеры одного фильтра, умноженные на количество слоев и на количество фильтров. В то время, как правая часть занимает память только во время работы, обученные веса являются ценностью и должны сохраняться на диск.

**Количество операций, которые нам потребуются** - это количество элементов на выходе, которые мы хотим получить, умноженное на количество операций. Для получения каждого элемента нужно свернуть фильтр, который имеет размер К×К с каждым слоем входа.
Все это будет учитываться при построении сетей большего размера.


### Размер рецептивного поля


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_8.png"  width="850">

Авторы стремились уменьшить количество параметров, используя следующую идею:
Допустим, мы свернули часть изображения с фильтром 5×5 (на слайде выше - квадрат синего цвета), получили одну активацию (одна свертка - одна активация).

Если же начать сворачивать ту же область с фильтром 3×3, то мы получим блок активаций (изображен красным). Но если второй раз применить такой же фильтр (размером 3×3) уже к блоку А2, он его закроет полностью и мы получим единичную активацию.

Для того же рецептивного поля свертка с двумя последовательными фильтрами 3×3 будет эквивалентна свертке с одним фильтром 5×5. При этом потребуется меньшее количество параметров, потому что в данном случае мы складываем две свертки. В случае же с одним фильтром 5×5 это зависит от размера фильтра, который возводится в квадрат. Соответственно, $2⋅(3^2⋅𝐶^2)$ меньше, чем $5^2⋅𝐶^2$.


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_9.png"  width="850">

Если же размер фильтра будет ещё больше, то будет больше и выгода.

Получается, что можно заменять фильтры, не теряя при этом их обобщающую способность. К тому же в процессе применения этих нескольких фильтров мы добавляем дополнительные нелинейности, которые нам обеспечивают находящиеся между свертками функции активации. То есть операция получается достаточно выигрышной.


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_10.png"  width="850">


Отчасти благодаря такой экономии получилось сделать большую по тем временам сеть (16 слоев), но, несмотря на способы уменьшить вычислительную сложность и количество параметров, сеть все равно получилась огромной (для 16-тислойной сети одно изображение в памяти занимало 96 МБ).

Надо понимать, что при обучении мы используем батч, размеры этого батча ограничены этим <font color=red >каким?</font> числом.


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_11.png"  width="850">


* Основная часть памяти расходуется на большие свертки в начальных слоях, где пространственные размеры(ширина и высота) велики

* Больше всего весов в полносвязанных слоях

* Вычислительные ресурсы нужны в первую очередь для сверток


VGG-16 получилась существенно больше по сравнению с и так довольно объемной AlexNet, и тем более по сравнению с современными моделями.

В значительной степени с этим связано дальнейшее направление развития моделей. В следующем году ImageNet выиграла сеть под названием GoogleNet.


## GooleNet
В отличие от предшествующих моделей разработанна в коммерческой компании, с целью реального применения. 

Поэтому оснвной упор бул сделан на эффективности.


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_12.png"  width="850">

GoogleNet — ещё более глубокая архитектура с 22 слоями.

В GoogleNet нет полносвязных слоёв, и она содержит всего 5 миллионов параметров — в 12 раз меньше, чем у AlexNet. При этом сеть оказалась немного более точной (ошибка снизилась с 7.3% до 6.7%).

Рассмотрим, за счет чего удалось достичь такого огромного выигрыша в ресурсах, так как многие идеи, которые впервые были использованы для GoogleNet, активно применяются до сих пор.


 ### Inception module

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_13.png"  width="850">

Для того, чтобы разработать с наибольшей вычислительной эффективностью, был придуман так называемый модуль Inception, основная идея которого заключается в том, что он сам по себе является небольшой локальной сетью. Его архитектура состоит из множества таких модулей, следующих друг за другом.

Вместо того, чтобы добавлять новые слои фильтров один над другим, инженеры из Google решили делать свертки параллельно. Они попробовали поместить разные варианты сверток в один блок и объединить их результаты (конкатенировать, а не сложить) между собой.

То есть фильтры применяются параллельном применении нескольких фильтров на исходное изображение. Затем данные фильтров объединяются, и создаётся выходной сигнал, который переходит на следующий слой.

Но здесь же кроется проблема: При таком подходе "в лоб" количество фильтров все равно растет довольно быстро.
Рассмотрим пример: возьмём упрощённую версию модуля Inception и попробуем посчитать размеры выходных изображений после применения каждого фильтра. При исходной фотографии 28×28×256 получим следующее:

<font color=red >(обсудить добавление картинки)</font>

В результате вычислений мы обнаружили, что наш небольшой блок выполняет 854 миллиона операций! При этом pooling-слои не уменьшают размер объекта — это означает, что после каждого объединения фильтров он будет только расти, и вычисления станут очень дорогостоящими.


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_14.png"  width="850">

Чтобы этого избежать, введены так называемые «узкие места» — слои с фильтром 1×1, уменьшающие глубину изображения. Благодаря им удалось достичь того, чтобы количество каналов на входе и на выходе либо не менялось, либо менялось только в моменты, когда это необходимо.

Как видно на изображении выше, для сверток с неединичным фильтром перед ними добавляется свертка размером 1×1, а max pooling ненастоящий, его шаг 1, он не меняет пространственные размеры, а просто выбирает свертку с максимальным значением из поля 3×3.
Такой слой работает более эффективно.


### 1x1 Convolution

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_15.png"  width="850">

*Уменьшение глубины изображения с помощью 32 фильтров 1×1*

Сверку 1х1 можно сравнить с полносвязанными слоями, когда мы берем столбик из этого тензора и сворачиваем его с некоторым количеством фильтров, получаем на выходе вектор (то есть получаем из одного вектора другой).

<font color=red >Попытка перевода: В качестве альтернативы, интерпретируйте это как применение того же FC слоя на каждом входном пикселе, сохраняющем пространственные размеры, уменьшающем глубину. Проецирует глубину до нижнего измерения (комбинация карт объектов)</font>

Alternatively, interpret it as applying the same FC layer on each input pixel preserves spatial dimensions, reduces depth! Projects depth to lower dimension (combination of feature maps)



<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_16.png"  width="850">


Количество параметров уменьшается в два с лишним раза по сравнению с лобовой реализацией. Сеть получается значительно экономичнее.

Использование таких модулей и отсутствие полносвязных слоёв делают GoogleNet очень эффективной и достаточно точной сетью. Но это далеко не все нововведения, которые появились в этой модели.


### "Stem network"

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_17.png"  width="850">


 - в первых слоях быстро уменьшаются пространственные размеры. 
 
 В составе GoogleNet есть небольшая подсеть — Stem Network. Она состоит из трёх свёрточных слоёв с двумя pooling-слоями и располагается в самом начале архитектуры.
 
На входе располагаются свертки с большим фильтром. Они служат для того, чтобы достаточно быстро и сильно уменьшить пространственные размеры (по сути сжать изображение перед параллельной обработкой), чтобы минимизировать количество элементов в слоях.

Отдельно стоит обратить внимание на завершающую часть сети: Global Average Pooling. Несложно заметить, что больше всего параметров появляется в полносвязных слоях, где каждый элемент связан с каждым. Сверточный слой очень экономичный по сравнению с полносвязным. В предыдущих моделях в конце было два полносвязных слоя, в которых находилась большая часть весов.

Была придумана достаточно интересная вещь: слой глобального пулинга (при котором берется среднее значение).


### Global Average Pooling

Полносвязанные слои замененны на GAP.

* Меньше весов
* Не зависимость от размера входа
* Регуляризация


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_18.png"  width="850">

<a href="https://arxiv.org/pdf/1312.4400.pdf">2014 Network In Network</a>

https://alexisbcook.github.io/2017/global-average-pooling-layers-for-object-localization/

Идея в том, что все пространственные размеры, какими бы они не были (например, 6х6), сворачиваются в единицу.
Мы накладываем фильтр размером 6х6 и берем среднее значение. То есть делаем пулинг с фильтром такого размера. Это происходит независимо по каждому слою.

Таким образом, мы избавляемся от пространственного измерения и получаем на выходе вектор. При этом нам не нужно запоминать кучу параметров, как в полносвязном слое, и таким образом, это выполняет некоторую регуляризацию, которая позволяет, в том числе, бороться с переобучением, потому что мы избавляемся от части менее важных активаций.


### Затухание градиента

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_19.png"  width="850">

Помимо основного классификатора на выходе сети, два дополнительных классификатора, встроенных в промежуточные слои. Они понадобились для того, чтобы улучшить обратное распространение градиента, потому что без батч-нормализации в таких глубоких сетях градиент очень быстро затухал, и обучить сеть такого размера было серьезной проблемой.

Обучение VGG осуществлялось непростым способом: сначала обучали 7 слоев, затем добавляли туда следующие и обучали это вручную. Без использования батч-нормализации вряд ли получится повторить результат.

Google подошел более системно, он добавил дополнительные выходы, которые способствовали тому, чтобы градиент меньше затухал. Благодаря этому удалось решить серьезную на тот момент проблему, которая ограничивала возможность обучения глубоких моделей.
Статья про батч-нормализацию появилась как раз в 15ом году, видимо уже после выхода этой модели.


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_20.png"  width="850">

В 15ом году появились существенно более глубокие модели. Но одной батч-нормализации было недостаточно, потому что появилась возможность благодаря ее использованию тренировать сети с большим количеством слоев (на слайде выше - пример 56-слойной сети Microsoft). Но, как видно из графиков, у нее ошибка и на тренировочном, и на тестовом датасете больше, чем у 20-тислойной сети.

## ResNet

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_21.png"  width="850">



<a href="https://arxiv.org/pdf/1512.03385.pdf">2015 Deep Residual Learning for Image Recognition</a>

В 2015 году соревнования выиграла сеть ResNet, произведя настоящую революцию глубины нейросетей. Она состояла из 152 слоёв и снизила процент ошибок до 3,57%. Это сделало её почти в два раза эффективнее GoogleNet.

Возникло предположение, что сеть, состоящая из большего количества слоев должна работать как минимум не хуже, чем сеть меньшего размера, потому что в ней есть те же 20 слоев.

Что же происходит с нейросетью, когда мы увеличиваем число слоёв? Можно ли, взяв обычную архитектуру вроде VGG, просто складывать всё больше и больше слоёв друг на друга и достигать лучшей точности?

Нет, нельзя. Скорее всего, более глубокая нейросеть покажет даже худшие результаты как при обучении, так и при тестировании. И переобучение здесь не при чём, поскольку тогда тренировочная ошибка была бы низкой.

Можно провести подобную аналогию: если мы видим изображение какое-то короткое время и нам нужно быстро его описать, мы скорее всего заметим какие-то крупные объекты (сцена: либо это помещение, либо лес, улица, возможно, мы отметим фигуры людей на переднем плане или же автомобили, здания. Если же требуется более подробно описать рассмотреть и описать картинку, мы уже к этому же имеющемуся описанию, добавим детали: во что одеты люди, какого цвета машины, погода и так далее.

Возможно, руководствуясь подобными соображениями, был придуман остаточный слой (residual).


Resudial connection

* сумма а не конкатенация
* Batch normalization присутствует


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_22.png"  width="850">

https://arxiv.org/pdf/1512.03385.pdf

Его идея состоит в том, что мы к имеющемуся уже набору признаков добавляем значения некоторых новых: то есть не перезаписываем то, что было на предыдущем слое, а копируем признаки, сохраняем их и через несколько слоев мы суммируем (а не конкатенируем) их с результатами сверток вот на нескольких слоях, которые здесь присутствуют (Residual block).

Все это вместе называется остаточным слоем или residual block.

Это дало потрясающий эффект: с одной стороны, по этому каналу стал хорошо распространяться градиент без дополнительных хаков, с другой стороны, перестали теряться важные свойства, которые удалось выделить на предыдущих блоках.

Если сеть из 20ти слоев способна распознавать с точностью более 90%, то там уже есть большая часть информации, а детали не должны ее затирать. Такая архитектура показала очень хороший результат.


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_23.png"  width="850">

identity =  Тождественное отображение

Из таких блоков удалось построить очень глубокую сеть (были эксперименты из 1000 слоев). Для решения конкретной задачи победы на ImageNet хватило 150 слоев (добавление большего количества блоков уже не давало прироста точности для данной задачи).

Здесь используются многие идеи, которые присутствовали в предыдущих моделях: вначале изображение резко уменьшается, дальше используются блоки 3х3, как в VGG, далее применяется глобальный пулинг вместо несколько полносвязных слоев. Блоки состоят из конструкций, изображенных выше: две свертки 3Х3 и прибавление результата предыдущего слоя.

Также, на каждом слое перед активацией используется батч-нормализация.

Периодически, в местах, где используется деление на 2, здесь в два раза уменьшаются пространственные размеры и также в 2 раза увеличивается количество фильтров.


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_24.png"  width="850">

Если посмотреть на реализацию ImageNet, то вот что происходит с размерами изображения: сначала его уменьшают обычной сверткой с большим фильтром, а потом, в зависимости от того, насколько глубокая сеть некоторое количество идет блоков без уменьшения, потом происходит даунсэмплинг, при том здесь используется не pulling, а просто свёртка с шагом 2.

Таким образом через некоторые набор вот этих блоков из изображений уменьшается в два раза вот на выходе average pooling. Даже вот такая огромная 152-хслойная сеть с точки зрения потребления ресурсов получилось более эффективной, чем VGG-19.


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_25.png"  width="850">

Это еще было достигнуто также за счет того, что в более глубоких сетях вместо двух блоков 3х3 применялся вот такой более эффективный блок, где сначала происходит свёртка 1х1 и мы уменьшаем количество фильтров. С этим маленьким количеством фильтров делаем свертку 3х3, а потом восстанавливаем количество фильтров до начальных значений, чтобы можно было прибавить к ним вход, иначе у нас не совпадут размерности. Засчет этого достигается эффективность.

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_26.png"  width="850">

https://pytorch.org/docs/stable/_modules/torchvision/models/resnet.html

Так выглядит в коде этот базовый блок для сетей со слоями до 50 слоев. То есть  свёртка, батч-нормализация, активация, свертка, батч-нормализация. Если некоторый параметр (даунсемпл) задан, то вызывается несколько слоев для даунсемплинга: то есть это конволюция (свертка) и ReLu с батч-нормализацией. Это реализуется простым прибавлением значения выходов к входам. 


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_27.png"  width="850">


<a href="https://arxiv.org/abs/1901.09321">2019 Fixup Initialization: Residual Learning Without Normalization</a>

https://towardsdatascience.com/understanding-fixup-initialization-6bf08d41b427

В дополнение можно заметить, что не так давно вышла статья, где автор обратили внимание, что правила, которые мы рассматривали для инициализации (что дисперсия входов должна равняться дисперсии выходов) нарушается при таком подходе, потому что мы здесь к выходам добавляем некоторые значения. Авторы  предлагают решить эту проблему довольно странным на первый взгляд способом - обнулить веса (2ого <font color=red >(???)</font>) слоя при инициализации. На первый взгляд это звучит как ошибка, но если вдуматься, поскольку (<font color=red >(сюда ????как это можно назвать???)</font>) мы добавляем значение c этого <font color=red >(???)</font> выхода, то при обратном распространении этот градиент не будет нулевым, потому что за счет входа (Х) значения появятся и, когда мы будем при обратном распространении спускаться сюда <font color=red >(???)</font> будем считать градиент по весам. Этот <font color=red >(???)</font> вход у нас тоже не будет нулевым, потому что здесь мы ничего не обнуляли. Градиенты будут равны входам по весам и за счет них мы получим какое-то обновление. 



<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_28.png"  width="850">

Обучение ResNet.

Когда точность выходила на плато, шаг обучения понижали вручную. Это давало хороший эффект: ошибка падала.


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_29.png"  width="850">

Помимо того, что эта модель c огромным отрывом выиграла ImageNet от моделей прошлого года и, как уже видно по некоторым исследованиям, ее результаты лучше, чем у человека, решения на базе этой архитектуры также стали победителями на соревнованиях по детектированию и сегментации. 


<font color=red >(???надо этот кусок???)</font> Почему вообще так много говорили о модели для классификации? Это важная задача, но далеко не единственная. Детектирование может даже интересней. <font color=red >(???)</font>


## Feature extraction
<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_30.png"  width="850">


Дело в том, что в процессе выполнения задачи классификации картинок у нас появляются веса, которые отвечают за распознавание разных особенностей, разных паттернов на изображениях. Эти веса в сверточных слоях представляют большую ценность, потому что мы можем их использовать в данном случае для классификации.


та же картинка + Х


Но для решения другой задачи мы можем часть сети просто отрезать и заменить ее чем-то еще:


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_31.png"  width="850">

Например, мы можем на основании карт активаций, которые мы получили (которые уже намного меньше, чем были начальные изображения) проводить детектирование в этих картах, проводить сегментацию, можем генерировать вектора признаков (embedding) для разных задач: для сравнения изображений, для распознавания лиц, трекинга и многого другого.

Часть сети (слева) является достаточно универсальной в сети и получить эти веса
можно, обучая сеть распознавать изображения для классификации, а потом использовать ещё  каким-то образом, возможно даже еще не придуманным в настоящий момент. 

Эта часть сети называется backbone (скелет, основа), на которые уже строятся какие-то дополнительные алгоритмы.


## Обзор моделей

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_32.png"  width="850">

https://arxiv.org/pdf/1810.00736.pdf

Данная картинка из статьи 18 года, то есть не последняя, но тем не менее, на них довольно наглядно собрано много информации: здесь есть информация и о размере модели, и о скорости, и о точности. Здесь можно увидеть, что VGG - огромные по объему модели, но по нынешним меркам они обладают средний точностью.

Они требуют больших вычислительных ресурсов, поэтому ну сейчас их имеет смысл использовать разве что в учебных целях, а модели на базе ResNet (ResNet-50, ResNet-152)  довольно хороши: в плане точности какого-то большого отрыва от них здесь не видно. Но, тем не менее, есть модели, которые работают несколько лучше. Рассмотрим их кратко, чтобы было понимание того, куда двигалась мысль в этой области.



## DensNet

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_33.png"  width="850">

Один из вариантов, что можно сделать - это добавить еще дополнительных связей в обход блоков, чтобы градиент тек еще лучше.

Можно заменить сумму на конкатенацию. Это тоже работает, но надо понимать, что конкатенация увеличивает количество признаков, поэтому надо с этим работать видимо с этим можно бороться засчет сверток 1Х1. Это так называемый DenceNet. С точки зрения ресурсов они, как правило, чуть более требовательные, чем базовый ResNet и немного более точный. 



## ResNeXt

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_34.png"  width="850">



<a href="https://arxiv.org/abs/1611.05431">2016 Aggregated Residual Transformations for Deep Neural Networks</a>

Еще одна идея - это ResNeXt. Эта сеть выиграла ImageNet в следующем году, а здесь налицо заимствование идеи от Inception модуля. Можно обрабатывать не сразу все каналы, а распараллелить обработку и обрабатывать по нескольку каналов. Эта идея уже была в AlexNet, но она там присутствовала вынужденно, потому что AlexNet обучали на двух видеокартах параллельно, просто в силу того, что модель не помещалась в память одной видеокарты на тот момент (видеокарты были по 3 Гб) и уже в конце на полносвязных слоях объединяли результаты двух карт, то есть там тоже обработка происходила параллельно вынужденным образом. 


### Groupped Convolution

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_35.png"  width="850">

https://towardsdatascience.com/a-comprehensive-introduction-to-different-types-of-convolutions-in-deep-learning-669281e58215

Это не так плохо, потому что это, с одной стороны и увеличивает скорость и число параметров получается меньше и в некотором смысле работает как регуляризация.

На изображении выше на AlexNet видно, что сеть у них разделена на 2 части. 

<font color=blue >(отд картинка нижняя часть)?</font>

Это демонстрация того, как это работает в обычном слое: можем просто каким-то образом разделить тензор так, что часть каналов начнет обрабатываться одной сверткой, а часть другой, а потом их просто конкатенируем. Глубина фильтров будет соответственно меньше, которые потребуются для этой свертки. 



<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_36.png"  width="850">

Можно считать параллельного. Не нужно дожидаться результатов предыдущих операций, можем параллельно их считать. Этот механизм уже заложен в свёртку, которой пользовались в pytorch, просто есть параметр группы по умолчанию он (???), то есть означает, что групп нет. Если его поменять, то у нас обработка будет происходить параллельно таким образом.

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_37.png"  width="850">

https://towardsdatascience.com/review-resnext-1st-runner-up-of-ilsvrc-2016-image-classification-15d7f17b42ac

Это используется и в inception сетях, но там результаты конкетинируются и в ResNeXt, и в Inception сетях, в (??Xt??), где они суммируются.

То есть эта операция групп convolutional оказалась достаточно эффективной. 


## WideResNet


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_38.png"  width="850">



<a href="https://arxiv.org/abs/1605.07146">2016 Wide Residual Networks</a>

dropout внутри блоков

Еще одна идея, связанная уже не с распараллеливанием, а с увеличением количества фильтров. Мы можем увеличивать количество фильтров и уменьшить количество слоев. В такого рода моделях первая цифра - это количество слоев, вторая - коэффициент, с которым мы увеличиваем количество наших фильтров. Поскольку эта операция тоже неплохо распараллеливается, 50-тислойная сеть с коэффициентом (??2 или 4??) обгоняет 150тислойную оригинальную ResNet. 


## SENet

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_39.png"  width="850">



<a href="https://arxiv.org/abs/1709.01507">2017 Squeeze-and-Excitation Networks</a>

https://towardsdatascience.com/squeeze-and-excitation-networks-9ef5e71eacd7

MobileNet
<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_40.png"  width="850">

### Depthwise separable convolution

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_41.png"  width="850">

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_42.png"  width="850">

### Shuffled Grouped Convolution

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_43.png"  width="850">

## Neural Architecture Search

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_44.png"  width="850">

## Visual Transformers

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_45.png"  width="850">




<a href="https://arxiv.org/abs/2006.03677">2020 Visual Transformers: Token-based Image Representation and Processing for Computer Vision</a>


old version: https://syncedreview.com/2020/06/12/facebook-and-uc-berkeley-boost-cv-performance-and-lower-compute-cost-with-visual-transformers/

MAC (multiply and accumulate operations) 


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_46.png"  width="850">